In [16]:
import shutil
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
def canny(gray_image,th):
    med_val = np.median(gray_image)
    lower = int(max(0,0.7*med_val))
    upper = int(min(255,1.3*med_val))
    return cv2.Canny(th,lower-15,upper+20)

def ROI(gray_image):
    height = gray_image.shape[0]
    width = gray_image.shape[1]
    mask = np.zeros_like(gray_image)
    mask[int(0.6*height):height,int(0.3*width):int(0.7*width)] = gray_image[int(0.6*height):height,int(0.3*width):int(0.7*width)]
    return mask
    
def make_lines(gray_image, colored_image):
    lines = cv2.HoughLinesP(gray_image, 2.5, np.pi/180, 110,np.array([]), maxLineGap=7,minLineLength=45)
    height = gray_image.shape[0]
    width = np.int32(gray_image.shape[1])
    line_image = np.zeros_like(colored_image)
    left_line = []
    right_line = []

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            slope,intercept = np.polyfit((x1,x2),(y1,y2),1)
            if np.absolute(slope)<0.3:
                continue
            if slope > 0:
                right_line.append((slope,intercept))
            else:
                left_line.append((slope,intercept))
 
        y=np.int32(height*2/3)
        if len(left_line)>0:
            left_line_slope_avg, left_line_intercept_avg = np.average(left_line,axis=0)
            cv2.line(line_image, (np.int32((height-left_line_intercept_avg)/left_line_slope_avg),np.int32(height)), (np.int32((y-left_line_intercept_avg)/left_line_slope_avg), np.int32(y)), (255, 0, 0), 15)
        
        if len(right_line)>0:
            right_line_slope_avg, right_line_intercept_avg = np.average(right_line,axis=0)
            cv2.line(line_image, (np.int32((height-right_line_intercept_avg)/right_line_slope_avg),np.int32(height)), (np.int32((y-right_line_intercept_avg)/right_line_slope_avg),np.int32(y)), (255, 0, 0), 15)

    return line_image
  

def shadow_remove(img):
    rgb_planes = cv2.split(img)
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 21)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_norm_planes.append(norm_img)
    shadowremov = cv2.merge(result_norm_planes)
    return shadowremov

In [18]:
base_dir = 'C:\\Users\\anak1\\Desktop\\project\\2011_09_26\\2011_09_26_drive_0028_sync\\image_02\\data'
image_array = []
for image in os.listdir(base_dir):
    image_path = os.path.join(base_dir,image)
    colored_img = cv2.imread(image_path)
    colored_copy = cv2.cvtColor(np.copy(colored_img),cv2.COLOR_BGR2RGB)
    gray_img = cv2.imread(image_path,0)
    sh = shadow_remove(gray_img)
    adjusted = cv2.convertScaleAbs(sh, alpha=0.77, beta=-71)
    ret,th = cv2.threshold(adjusted,adjusted.max()-30,255,cv2.THRESH_BINARY_INV)
    res = cv2.GaussianBlur(th,(17,17),0)
    cannyed = canny(gray_img,res)
    roied = ROI(cannyed)
    line_img = make_lines(roied,colored_copy)
    blended = cv2.addWeighted(src1=colored_copy,alpha=0.8,src2=line_img,beta=1.0,gamma=1)
    image_array.append(blended)
size = (gray_img.shape[1],gray_img.shape[0])       
out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(image_array)):
    out.write(cv2.cvtColor(np.copy(image_array[i]),cv2.COLOR_RGB2BGR))
out.release()
    
    